## Import Library

In [1]:
import numpy as np
import cv2
import matplotlib.image as mpimg
import PIL
import json
import os
import time

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from PIL import Image
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA

## PATH

In [41]:
VID_PATH = "E:\\Ari Folder\\Own folder\\Dokumen\\Kuliah UB\\Semester6\\KCD\\Media\\Video\\H112\\H112_2.mp4"
CROP_PATH = "E:\\Ari Folder\\Own folder\\Dokumen\\Kuliah UB\\Semester6\\KCD\\Media\\Image\\H112\\"
DATASET_PATH = r"E:/Ari Folder/Own folder/Dokumen/Kuliah UB/Semester6/KCD/Media/Image/character/test/"

## Ektraksi Frame Video

In [ ]:
cam = cv2.VideoCapture(VID_PATH)

try :
    #bikin folder
    if not os.path.exists('data'):
        os.makedirs('data')
except OSError :
    print ('Error while creating folder')
    
currentframe = 0

while(True):
    #baca semua frame
    ret, frame = cam.read()
    
    if ret :
        #kalo durasi video belum habis, maka tetep ekstrak gambar
        name = './data/frame'+str(currentframe)+'.jpg'
        print('Creating :'+name)
        
        #tulis gambar yang diekstrak
        cv2.imwrite(name,frame)
        
        #counter naik seiring banyaknya frame yang telah diproses
        currentframe += 1
    else :
        break

#lepaskan memori dan tutup semua windows begitu kelar
cam.release()
cv2.destroyAllWindows()

## Grayscale & Crop

In [ ]:
CROP_PATH = "E:\\Ari Folder\\Own folder\\Dokumen\\Kuliah UB\\Semester6\\KCD\\Media\\Image\\H112\\"

try :
    #bikin folder
    if not os.path.exists('crop'):
        os.makedirs('crop')
except OSError :
    print ('Error while creating folder')

currentimg = 0

y = 250
x = 300
h = 300
w = 600

while(True):
    img_path = CROP_PATH+'frame'+str(currentimg)+'.jpg'
    #cek file ada atw engga
    if os.path.exists(img_path):
        img = cv2.imread(img_path,0)
            
        crop_img = img[y:y+h, x:x+w]
            
        name = './crop/frame'+str(currentimg)+'.jpg'
        cv2.imwrite(name, crop_img)
            
        print ('Creating : '+name)
        currentimg += 1
        
        #MATIKAN ini bila ingin crop semua gambar, UBAH ini bila hanya ingin crop sampai gambar tertentu
        #if currentimg == 1395 : break
        #####
    else :
        print ('No file exist in :'+img_path)
        break

cv2.destroyAllWindows()

## Fungsi LBP

In [3]:
def get_pixel(img, center, x, y):
      
    new_value = 0
      
    try:
        # If local neighbourhood pixel 
        # value is greater than or equal
        # to center pixel values then 
        # set it to 1
        if img[x][y] >= center:
            new_value = 1
              
    except:
        # Exception is required when 
        # neighbourhood value of a center
        # pixel value is null i.e. values
        # present at boundaries.
        pass
      
    return new_value
   
# Function for calculating LBP
def lbp_calculated_pixel(img, x, y):
   
    center = img[x][y]
   
    val_ar = []
      
    # top_left
    val_ar.append(get_pixel(img, center, x-1, y-1))
      
    # top
    val_ar.append(get_pixel(img, center, x-1, y))
      
    # top_right
    val_ar.append(get_pixel(img, center, x-1, y + 1))
      
    # right
    val_ar.append(get_pixel(img, center, x, y + 1))
      
    # bottom_right
    val_ar.append(get_pixel(img, center, x + 1, y + 1))
      
    # bottom
    val_ar.append(get_pixel(img, center, x + 1, y))
      
    # bottom_left
    val_ar.append(get_pixel(img, center, x + 1, y-1))
      
    # left
    val_ar.append(get_pixel(img, center, x, y-1))
       
    # Now, we need to convert binary
    # values to decimal
    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
   
    val = 0
      
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
          
    return val

## Ekstraksi FAST, LBP, Resize, Padding, dan Pemberian Label

In [42]:
min_len = 38
max_len = 336

fast_dic = {
    "label": [],
    "kp": []
}

start = time.time()
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(DATASET_PATH)):
    if dirpath is not DATASET_PATH:
        label = dirpath.split("/")[-1]
        print("\nProcessing: '{}'".format(label))
        
    for f in filenames :
        file_path = os.path.join(dirpath, f)
        img_file = np.array(cv2.imread(file_path,0))
        
        #ekstraksi fitur FAST
        fast = cv2.FastFeatureDetector_create(10)
        kp = fast.detect(img_file, None)
        pts = np.array([p.pt for p in kp]).flatten()
        
        #tandai keypoint pada file gambar
#         img_fast = cv2.drawKeypoints(img_file, kp, None, color=(255,0,0))
#         name = str(i-1)+str(f)+'.jpg'
#         cv2.imwrite(name, img_fast)
        
        #hitung jumlah fitur terbanyak dan terkecil
        length = len(pts)
        if(length > max_len) : max_len = length
        if(length < min_len) : min_len = length
        
        #padding
        num_missing_items = max_len - len(pts)
        padded_pt = np.pad(pts, (0,num_missing_items),mode="constant")
        
        #Resize
#         img_rez = np.array(cv2.resize(cv2.imread(file_path,0),[50,50]))
        
        #Pengambilan Fitur Local Binary Pattern
#         height, width = img_rez.shape
#         img_lbp = np.zeros((height, width),np.uint8)
#         for j in range(0, height):
#             for k in range(0, width):
#                 img_lbp[j, k] = lbp_calculated_pixel(img_file, j, k)
        
        
        fast_dic["label"].append(i-1)
        fast_dic["kp"].append(padded_pt.flatten().tolist())
#         fast_dic["lbp"].append(img_lbp.flatten())
        print("{}: {}".format(file_path, i-1))


end = time.time()
total_time = end - start
print("extraction time: {}".format(total_time))

with open("fast_test.json", "w") as fp :
    json.dump(fast_dic, fp, indent=4)


Processing: 'G11'
E:/Ari Folder/Own folder/Dokumen/Kuliah UB/Semester6/KCD/Media/Image/character/test/G11\frame100.jpg: 0
E:/Ari Folder/Own folder/Dokumen/Kuliah UB/Semester6/KCD/Media/Image/character/test/G11\frame91.jpg: 0
E:/Ari Folder/Own folder/Dokumen/Kuliah UB/Semester6/KCD/Media/Image/character/test/G11\frame92.jpg: 0
E:/Ari Folder/Own folder/Dokumen/Kuliah UB/Semester6/KCD/Media/Image/character/test/G11\frame93.jpg: 0
E:/Ari Folder/Own folder/Dokumen/Kuliah UB/Semester6/KCD/Media/Image/character/test/G11\frame94.jpg: 0
E:/Ari Folder/Own folder/Dokumen/Kuliah UB/Semester6/KCD/Media/Image/character/test/G11\frame95.jpg: 0
E:/Ari Folder/Own folder/Dokumen/Kuliah UB/Semester6/KCD/Media/Image/character/test/G11\frame96.jpg: 0
E:/Ari Folder/Own folder/Dokumen/Kuliah UB/Semester6/KCD/Media/Image/character/test/G11\frame97.jpg: 0
E:/Ari Folder/Own folder/Dokumen/Kuliah UB/Semester6/KCD/Media/Image/character/test/G11\frame98.jpg: 0
E:/Ari Folder/Own folder/Dokumen/Kuliah UB/Semester6/

## Normalisasi

In [27]:
keypoints = np.array(fast_dic["kp"])
# lbps = np.array(fast_dic["lbp"])

norm_pts = (keypoints - keypoints.min()) / (keypoints.max() - keypoints.min())
norm_pts = norm_pts * (1 - 0) + 0
# norm_lbps = (lbps - lbps.min()) / (lbps.max() - lbps.min())
# norm_lbps = norm_lbps * (1 - 0) + 0

fast_dic["kp"] = norm_pts
# fast_dic["lbp"] = norm_lbps

## Split Data

In [13]:
x = np.array(fast_dic["kp"])
y = np.array(fast_dic["label"])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

In [ ]:
x_lbp = np.array(fast_dic["lbp"])
y_lbp = np.array(fast_dic["label"])

x_train_lbp, x_test_lbp, y_train_lbp, y_test_lbp = train_test_split(x_lbp, y_lbp, test_size=0.1)

## Simpan Data ke JSON

In [14]:
train = {
    "label": [],
    "kp": []
}
test = {
    "label": [],
    "kp": []
}

train["label"] = y_train.tolist()
train["kp"] = x_train.tolist()

test["label"] = y_test.tolist()
test["kp"] = x_test.tolist()

with open("train.json", "w") as fp :
    json.dump(train, fp, indent=4)
    
with open("test.json", "w") as fp :
    json.dump(test, fp, indent=4)

In [ ]:
train = {
    "label": [],
    "lbp": []
}
test = {
    "label": [],
    "lbp": []
}

train_lbp["label"] = y_train_lbp.tolist()
train_lbp["lbp"] = x_train_lbp.tolist()

test_lbp["label"] = y_test_lbp.tolist()
test_lbp["lbp"] = x_test_lbp.tolist()

with open("train_lbp.json", "w") as fp :
    json.dump(train, fp, indent=4)
    
with open("test_lbp.json", "w") as fp :
    json.dump(test, fp, indent=4)

In [2]:
TRAIN_PATH = "fast_train.json"
TEST_PATH = "fast_test.json"
f_train = open(TRAIN_PATH)
f_test = open(TEST_PATH)

train_data = json.load(f_train)
test_data = json.load(f_test)

x_train = np.array(train_data["kp"])
y_train = np.array(train_data["label"])

x_test = np.array(test_data["kp"])
y_test = np.array(test_data["label"])

## Modiel KNN & Pengujian

In [5]:
knn = neighbors.KNeighborsClassifier(n_neighbors=3, weights='uniform')
knn.fit(x_train,y_train)
predictionKNN = knn.predict(x_test)
accuracyKNN = metrics.accuracy_score(y_test, predictionKNN)
print('[[prediction KNN]]')
print('prediction: ', predictionKNN)
print('actual: ', y_test)
print('accuracy: ', accuracyKNN)

[[prediction KNN]]
prediction:  [0 0 0 3 4 2 0 2 0 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3
 3 3 3 3 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5]
actual:  [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3
 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5]
accuracy:  0.9


In [49]:
print('[[prediction KNN]]')

for i in range(len(x_test)) :
    start = time.time()
    one_predictionKNN = knn.predict(x_test[i].reshape(1, -1))
    print('prediction: ', one_predictionKNN)
    print('actual: ', y_test[i])
    end = time.time()
    total_time = end - start
    print('time: {} \n'.format(total_time))


[[prediction KNN]]
prediction:  [3]
actual:  0
time: 0.004000663757324219 

prediction:  [1]
actual:  0
time: 0.0039997100830078125 

prediction:  [4]
actual:  0
time: 0.004000425338745117 

prediction:  [3]
actual:  0
time: 0.0029990673065185547 

prediction:  [4]
actual:  0
time: 0.002997875213623047 

prediction:  [0]
actual:  0
time: 0.0029973983764648438 

prediction:  [1]
actual:  0
time: 0.0030040740966796875 

prediction:  [2]
actual:  0
time: 0.003998279571533203 

prediction:  [0]
actual:  0
time: 0.002995729446411133 

prediction:  [0]
actual:  0
time: 0.004998207092285156 

prediction:  [1]
actual:  1
time: 0.0019991397857666016 

prediction:  [1]
actual:  1
time: 0.003998756408691406 

prediction:  [1]
actual:  1
time: 0.0030045509338378906 

prediction:  [1]
actual:  1
time: 0.0029935836791992188 

prediction:  [1]
actual:  1
time: 0.00499725341796875 

prediction:  [1]
actual:  1
time: 0.012997150421142578 

prediction:  [1]
actual:  1
time: 0.012998580932617188 

predic